In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
try:
    train_df = pd.read_csv('/kaggle/input/fraud-detection/fraudTrain.csv')
    test_df = pd.read_csv('/kaggle/input/fraud-detection/fraudTest.csv')
except:
    train_df = pd.read_csv('fraudTrain.csv')
    test_df = pd.read_csv('fraudTest.csv')
train_df.head()




In [ ]:
train_df.isnull().sum()


- **Unnamed: 0:** ترتيب الصفوف (عديم الفائدة).
- **trans_date_trans_time:** تاريخ ووقت المعاملات.
- **cc_num:** رقم البطاقة الائتمانية.
- **merchant:** معلومات عن التاجر.
- **category:** التصنيف.
- **amt:** المبلغ المالي للمعاملة.
- **first:** الاسم الأول للشخص القام بالمعاملة.
- **last:** الاسم الأخير للشخص القام بالمعاملة.
- **gender:** الجنس.
- **street:** اسم الشارع.
- **city:** اسم المدينة.
- **state:** اسم الولاية أو المقاطعة.
- **zip:** الرمز البريدي.
- **lat:** خط العرض.
- **long:** خط الطول.
- **city_pop:** عدد السكان في المدينة.
- **job:** الوظيفة.
- **dob:** تاريخ الميلاد.
- **trans_num:** رقم المعاملة.
- **unix_time:** الوقت في نظام يونكس.
- **merch_lat:** خط العرض للتاجر.
- **merch_long:** خط الطول للتاجر.
- **is_fraud:** هل هو احتيالي أم لا.



In [ ]:
def data_pre(X):
    del_col=['merchant','first','last','street','zip','unix_time','Unnamed: 0','trans_num','cc_num']
    X.drop(columns=del_col,inplace=True)
   
    
    X['trans_date_trans_time']=pd.to_datetime(X['trans_date_trans_time'])
    X['trans_date']=X['trans_date_trans_time'].dt.strftime('%Y-%m-%d')
    X['trans_date']=pd.to_datetime(X['trans_date'])
    
    
    X['dob']=pd.to_datetime(X['dob'])
    
    #Calculate Age of each trans 
    X["age"] = (X["trans_date"] - X["dob"]).dt.days //365

    
    X['trans_month']=X['trans_date'].dt.month
    X['trans_year']=X['trans_date'].dt.year
    
    X['gender']=X['gender'].apply(lambda x : 1 if x=='M' else 0)
    X['gender']=X['gender'].astype(int)
    X['lat_dis']=abs(X['lat']-X['merch_lat'])
    X['long_dis']=abs(X['long']-X['merch_long'])
    X=pd.get_dummies(X,columns=['category'])
    X=X.drop(columns=['city','trans_date_trans_time','state','job','merch_lat','merch_long','lat','long','dob','trans_date'])
    return X
    
    
    

    
    



    

    

In [ ]:
train_df_pre=data_pre(train_df.copy())
train_df_pre.head()

In [ ]:
x_train=train_df_pre.drop('is_fraud',axis=1)
y_train=train_df_pre['is_fraud']

In [ ]:
test_df_pre=data_pre(test_df.copy())
test_df_pre.head()
x_test=test_df_pre.drop('is_fraud',axis=1)
y_test=test_df_pre['is_fraud']

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
from sklearn.preprocessing import StandardScaler

# Step 1: Fit the StandardScaler on the training data
scaler = StandardScaler()
scaler.fit(x_train)
x_train=scaler.transform(x_train)
x_test=scaler.transform(x_test)


In [ ]:
logistic_regression=LogisticRegression()
logistic_regression.fit(x_train,y_train)
y_pred_logistic = logistic_regression.predict(x_test)
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
accuracy_logistic

In [ ]:
DecisionTree=DecisionTreeClassifier()
DecisionTree.fit(x_train,y_train)
y_pred_dt = DecisionTree.predict(x_test)
accuracy_dt = accuracy_score(y_test, y_pred_dt)
accuracy_dt

In [ ]:
random_forest = RandomForestClassifier(random_state=42,n_estimators=100)
random_forest.fit(x_train, y_train)
y_pred_rf = random_forest.predict(x_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
accuracy_rf

In [ ]:
# Print classification report for each model
print("\nClassification Report for Logistic Regression:\n", classification_report(y_test, y_pred_logistic))
print("\nClassification Report for Decision Tree:\n", classification_report(y_test, y_pred_dt))
print("\nClassification Report for Random Forest:\n", classification_report(y_test, y_pred_rf))

In [ ]:
"""THX """
